In [ ]:
import os
import sys
import pandas as pd
import yaml 
from matplotlib import pyplot as plt
from matplotlib import ticker as mticker
import statsmodels.api as sm
import numpy as np
from itertools import product
import subprocess

with open("../../config.yaml.local", "r") as f:
    LOCAL_CONFIG = yaml.safe_load(f)
with open("../../config.yaml", "r") as f:
    CONFIG = yaml.safe_load(f)
sys.path.append("../python")

import globals
import data_tools as dt
import writing_tools as wt

LOCAL_PATH = LOCAL_CONFIG["LOCAL_PATH"]
RAW_DATA_PATH = LOCAL_CONFIG["RAW_DATA_PATH"]
DATA_PATH = LOCAL_CONFIG["DATA_PATH"]
R_PATH = LOCAL_CONFIG["R_PATH"]

RUN_R_SCRIPTS = False

RESULTS = {}

In [ ]:
# Output the data for regressions in R

qual_df = dt.get_post_quality_analysis_data()
qual_df.to_parquet(os.path.join(DATA_PATH, "post_quality_analysis_data.parquet"))

quant_df = dt.get_post_quantity_analysis_data()
quant_df.to_parquet(os.path.join(DATA_PATH, "post_quantity_analysis_data.parquet"))

In [ ]:
res = subprocess.run([R_PATH, LOCAL_PATH + "/src/R/pay-to-post-analysis.R"], check=True, capture_output=True, text=True)
print(res.stdout)

In [33]:
quantity_reg_coefs = pd.read_csv(
    os.path.join(DATA_PATH, 'posts_cost_reg.csv'),
    index_col = 0
)
coef = quantity_reg_coefs.loc['log_cost', 'coef(rquan3)']
elast = fr"{coef:.3f}"
double_effect = np.exp(coef*np.log(2)) - 1
double_effect_pct = fr"{np.abs(double_effect)*100:.1f}"
RESULTS['PostsCostElasticity'] = elast
RESULTS['PostsCostDoublingEffect'] = double_effect_pct
wt.update_results(RESULTS)


{'PostsCostElasticity': '-0.244',
 'PostsCostDoublingEffect': '15.6',
 'Zaps48CostElasticity': '0.187',
 'Zaps48CostDoublingEffect': '13.8'}

In [34]:
quality_reg_coefs = pd.read_csv(
    os.path.join(DATA_PATH, 'sats48_cost_reg.csv'),
    index_col = 0
)
coef = quality_reg_coefs.loc['log_cost', 'coef(rzaps3)']
elast = fr"{coef:.3f}"
double_effect = np.exp(coef*np.log(2)) - 1
double_effect_pct = fr"{np.abs(double_effect)*100:.1f}"
RESULTS['Zaps48CostElasticity'] = elast
RESULTS['Zaps48CostDoublingEffect'] = double_effect_pct
wt.update_results(RESULTS)


{'PostsCostElasticity': '-0.244',
 'PostsCostDoublingEffect': '15.6',
 'Zaps48CostElasticity': '0.187',
 'Zaps48CostDoublingEffect': '13.8'}

In [35]:
quality_reg_coefs = pd.read_csv(
    os.path.join(DATA_PATH, 'comments48_cost_reg.csv'),
    index_col = 0
)
coef = quality_reg_coefs.loc['log_cost', 'coef(rcomm3)']
elast = fr"{coef:.3f}"
double_effect = np.exp(coef*np.log(2)) - 1
double_effect_pct = fr"{np.abs(double_effect)*100:.1f}"
RESULTS['Comments48CostElasticity'] = elast
RESULTS['Comments48CostDoublingEffect'] = double_effect_pct
wt.update_results(RESULTS)


{'PostsCostElasticity': '-0.244',
 'PostsCostDoublingEffect': '15.6',
 'Zaps48CostElasticity': '0.187',
 'Zaps48CostDoublingEffect': '13.8',
 'Comments48CostElasticity': '0.033',
 'Comments48CostDoublingEffect': '2.3'}